In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))
    if len(filenames) > 3:
        print("...")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/captcha-hacker/sample_submission.csv
/kaggle/input/captcha-hacker/test/task1/x4LPcV5n6IvLj4vz.png
/kaggle/input/captcha-hacker/test/task1/W88fVMlAs5IpsXn4.png
/kaggle/input/captcha-hacker/test/task1/ZWDL6pUMfPu5c9jh.png
...
/kaggle/input/captcha-hacker/test/task2/jMalnsI5a5IWxYAi.png
/kaggle/input/captcha-hacker/test/task2/ihE9HHgyOINGEMcO.png
/kaggle/input/captcha-hacker/test/task2/ZATEVW3P5s0akZjd.png
...
/kaggle/input/captcha-hacker/test/task3/cXBlxYfvQWbiK7dn.png
/kaggle/input/captcha-hacker/test/task3/5gEp1jR9jNNfuqlk.png
/kaggle/input/captcha-hacker/test/task3/hEQ0WQtB9B7j8C2f.png
...
/kaggle/input/captcha-hacker/train/annotations.csv
/kaggle/input/captcha-hacker/train/task1/H85RQ6dbWUvLSIDV.png
/kaggle/input/captcha-hacker/train/task1/n2GC8uY1N4QfvVxe.png
/kaggle/input/captcha-hacker/train/task1/XOqfRx2R6SnoEjFr.png
...
/kaggle/input/captcha-hacker/train/task2/Mr4B2zxXk92hyzn9.png
/kaggle/input/captcha-hacker/train/task2/SIuRCnlK8VS91FhX.png
/kaggle/input/captcha-h

In [22]:
import csv
import cv2
import numpy as np
import random
import os
import time
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms, utils, io
# !pip install GPUtil
# from GPUtil import showUtilization as gpu_usage
# gpu_usage()         

| ID | GPU | MEM |
------------------
|  0 |  0% | 14% |


In [23]:
TRAIN_PATH = "/kaggle/input/captcha-hacker/train"
TEST_PATH = "/kaggle/input/captcha-hacker/test"
# device = "cpu"
device = "cuda" 
# and change your settings/accelerator to GPU if you want it to run faster

In [24]:
class Task1Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task1")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
#         img = cv2.imread(f"{self.root}/{filename}")
        img = io.read_image(f"{self.root}/{filename}")
#         img = cv2.resize(img, (32, 3))
#         img = np.mean(img, axis=2)
        if self.return_filename:
            return torch.FloatTensor(img / 256), filename
        else:
            return torch.FloatTensor(img / 256), int(label)

    def __len__(self):
        return len(self.data)

In [25]:
# class Model(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(1024, 512),
#             nn.LeakyReLU(),
#             nn.Linear(512, 10)
#         )
        
        
#     def forward(self, x):
#         b, h, w = x.shape
#         x = x.view(b, h*w)
#         return self.layers(x)

class ResNet18_1(nn.Module):
    def __init__(self):
        super(ResNet18_1, self).__init__()

        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 10)

    def forward(self, x):
        return self.model(x)

In [26]:
train_data = []
val_data = []

with open(f'{TRAIN_PATH}/annotations.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        if random.random() < 0.7:
            train_data.append(row)
        else:
            val_data.append(row)

train_ds = Task1Dataset(train_data, root=TRAIN_PATH)
train_dl = DataLoader(train_ds, batch_size=200, num_workers=4, drop_last=True, shuffle=True)

val_ds = Task1Dataset(val_data, root=TRAIN_PATH)
val_dl = DataLoader(val_ds, batch_size=200, num_workers=4, drop_last=False, shuffle=False)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [27]:
model1 = ResNet18_1().to(device)

torch.cuda.empty_cache()
# print(model)
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
image_change = transforms.Compose([transforms.Resize((384, 288)), transforms.Normalize(mean, std)])

acc_final = 0
for epoch in range(30):
    print(f"Epoch [{epoch}]")
    model1.train()
    for image, label in train_dl:
        image = image_change(image)
        image = image.to(device)
        label = label.to(device)
#         print("Shape of X [N, C, H, W]: ", image.shape)
        pred = model1(image)
        loss = loss_fn(pred, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del image, label
        
    sample_count = 0
    correct_count = 0
    model1.eval()
    with torch.no_grad():
        for image, label in val_dl:
            image = image_change(image)
            image = image.to(device)
            label = label.to(device)
#             print(label)

            pred = model1(image)
            loss = loss_fn(pred, label)

            pred = torch.argmax(pred, dim=1)

            sample_count += len(image)
            correct_count += (label == pred).sum()
            
        del image, pred
        
    print("accuracy (validation):", correct_count / sample_count)
    cur = correct_count / sample_count
    if cur >= 0.99 :
#         print(cur)
        torch.save(model1.state_dict(), "/kaggle/working/model_task1")

Epoch [0]
accuracy (validation): tensor(0.7140, device='cuda:0')
Epoch [1]
accuracy (validation): tensor(0.8951, device='cuda:0')
Epoch [2]
accuracy (validation): tensor(0.9729, device='cuda:0')
Epoch [3]
accuracy (validation): tensor(0.9915, device='cuda:0')
Epoch [4]
accuracy (validation): tensor(0.9915, device='cuda:0')
Epoch [5]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [6]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [7]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [8]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [9]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [10]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [11]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [12]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [13]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [14]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [15]
accuracy (validation): tensor(1., device='cuda:0'

In [28]:
# torch.cuda.empty_cache()
# test_data = []

# with open(f'{TEST_PATH}/../sample_submission.csv', newline='') as csvfile:
#     for row in csv.reader(csvfile, delimiter=','):
#         test_data.append(row)

# test_ds = Task1Dataset(test_data, root=TEST_PATH, return_filename=True)
# test_dl = DataLoader(test_ds, batch_size=500, num_workers=4, drop_last=False, shuffle=False)


# if os.path.exists('submission.csv'):
#     csv_writer = csv.writer(open('submission.csv', 'a', newline=''))
# else:
#     f = open('submission.csv', 'w', newline='')
#     csv_writer = csv.writer(f)
#     csv_writer.writerow(["filename", "label"])


# model.eval()
# with torch.no_grad():
#     for image, filenames in test_dl:
#         image = image_change(image)
#         image = image.to(device)

#         pred = model(image)
#         pred = torch.argmax(pred, dim=1)

#         for i in range(len(filenames)):
#             csv_writer.writerow([filenames[i], str(pred[i].item())])
#             time.sleep( 0.001 )
            
#     del image, pred

# # for filename, _ in test_data:
# #     if filename.startswith("task2") or filename.startswith("task3"):
# #         csv_writer.writerow([filename, 0])
#     f.close()


In [29]:
gpu_usage()
torch.cuda.empty_cache()
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
alphabet = 'abcdefghijklmnopqrstuvwxyz0123456789'
alphabets = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','0','1','2','3','4','5','6','7','8','9']
alpha = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
dic={'alphabet':alphabets}
labelencoder = LabelEncoder()
data_le=pd.DataFrame(dic)
data_le['alphabet'] = labelencoder.fit_transform(data_le['alphabet'])
data_le
onehotencoder = OneHotEncoder()
data_str_ohe=onehotencoder.fit_transform(data_le).toarray()
pd.DataFrame(data_str_ohe)
# data_dum = pd.get_dummies(data_le)
# pd.DataFrame(data_dum)

| ID | GPU | MEM |
------------------
|  0 | 65% | 98% |


,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
class Task2Dataset(Dataset):
    def __init__(self, data, root, transform=None, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task2")]
        print(len(self.data))
#         self.new_data = make_dataset(self.data, alphabet)
#         print(len(self.new_data))
        self.return_filename = return_filename
        self.root = root
        self.transform = transform
#       self.samples = make_dataset(self.data_path,self.ans_path,self.alphabet)
    
    def __getitem__(self, index):
        filename, label = self.data[index]
#         img = cv2.imread(f"{self.root}/{filename}")
        img = io.read_image(f"{self.root}/{filename}")
#         img = cv2.resize(img, (32, 3))
#         img = np.mean(img, axis=2)s
        if self.return_filename:
            return torch.FloatTensor(img / 256), filename
        else:
#             label = torch.Tensor(label)
            t=[]
            for char in str(label):
                v = [0] * 36
                v[alphabet.find(char)] = 1
                t += v
            return torch.FloatTensor(img / 256), torch.Tensor(t)
    def __len__(self):
        return len(self.data)

In [31]:
train_data = []
val_data = []

transfer = transforms.Compose([torch.Tensor()])

with open(f'{TRAIN_PATH}/annotations.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        if random.random() < 0.7:
            train_data.append(row)
        else:
            val_data.append(row)

train_ds = Task2Dataset(train_data, root=TRAIN_PATH, transform=transfer)
train_dl = DataLoader(train_ds, batch_size=200, num_workers=4, drop_last=True, shuffle=True)

val_ds = Task2Dataset(val_data, root=TRAIN_PATH, transform=transfer)
val_dl = DataLoader(val_ds, batch_size=200, num_workers=4, drop_last=False, shuffle=False)

1778
722


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [32]:
class ResNet18_2(nn.Module):
    def __init__(self):
        super(ResNet18_2, self).__init__()

        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 72)

    def forward(self, x):
#         b, h, w = x.shape
#         x = x.view(b, h*w)
# #         return self.model(x)
        return self.model(x)

In [33]:
model2 = ResNet18_2().to(device)

torch.cuda.empty_cache()
# print(model)
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
criterion = nn.MultiLabelSoftMarginLoss()
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
image_change = transforms.Compose([transforms.Resize((384, 288)), transforms.Normalize(mean, std)])

for epoch in range(70):
    print(f"Epoch [{epoch}]")
    model2.train()
    for image, label in train_dl:
        image = image_change(image)
        image = image.to(device)
        label = label.to(device)
#         print(label.size())
#         print("Shape of X [N, C, H, W]: ", image.shape)
        pred = model2(image)
#         print(pred.size())
        loss = criterion(pred, label)
#         loss = loss_fn(pred, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del image, label
        
    sample_count = 0
    correct_count = 0
    acc_history = []
    acc_final = 0
    model2.eval()
    with torch.no_grad():
        for image, label in val_dl:
            image = image_change(image)
            image = image.to(device)
            label = label.to(device)

            pred = model2(image)
            loss = criterion(pred, label)
#             loss = loss_fn(pred, label)
            
#             pred = torch.argmax(pred, dim=1)
            
            
#             sample_count += len(image)
#             correct_count += (label == pred).sum()
#             acc = calculat_acc(pred, label)
            pred, label = pred.view(-1, 36), label.view(-1, 36)
            pred = torch.argmax(pred, dim=1)
            label = torch.argmax(label, dim=1)
            pred, label = pred.view(-1, 2), label.view(-1, 2)
            correct = []
            for i, j in zip(pred, label):
                if torch.equal(i, j):
                    correct.append(1)
                else:
                    correct.append(0)
            acc = sum(correct) / len(correct)
            acc_history.append(float(acc))
        del image, pred
    print("accuracy (validation):",  sum(acc_history) / len(acc_history))
#     print("accuracy (validation):", torch.mean(torch.Tensor(acc_history)))
    cur = sum(acc_history) / len(acc_history)
    if cur >= 0.99 :
#         print(cur)
        torch.save(model2.state_dict(), "/kaggle/working/model_task2")

Epoch [0]
accuracy (validation): 0.0
Epoch [1]
accuracy (validation): 0.00375
Epoch [2]
accuracy (validation): 0.021147540983606557
Epoch [3]
accuracy (validation): 0.11788934426229508
Epoch [4]
accuracy (validation): 0.4511065573770492
Epoch [5]
accuracy (validation): 0.7587704918032787
Epoch [6]
accuracy (validation): 0.8479098360655738
Epoch [7]
accuracy (validation): 0.9261065573770492
Epoch [8]
accuracy (validation): 0.9675
Epoch [9]
accuracy (validation): 0.9776024590163934
Epoch [10]
accuracy (validation): 0.9846516393442624
Epoch [11]
accuracy (validation): 0.9884016393442623
Epoch [12]
accuracy (validation): 0.9904508196721311
Epoch [13]
accuracy (validation): 0.9879508196721312
Epoch [14]
accuracy (validation): 0.99125
Epoch [15]
accuracy (validation): 0.9892008196721311
Epoch [16]
accuracy (validation): 0.995
Epoch [17]
accuracy (validation): 0.9924999999999999
Epoch [18]
accuracy (validation): 0.9942008196721313
Epoch [19]
accuracy (validation): 0.9917008196721312
Epoch [20

In [34]:
# torch.cuda.empty_cache()
# test_data = []

# with open(f'{TEST_PATH}/../sample_submission.csv', newline='') as csvfile:
#     for row in csv.reader(csvfile, delimiter=','):
#         test_data.append(row)

# test_ds = Task2Dataset(test_data, root=TEST_PATH, return_filename=True)
# test_dl = DataLoader(test_ds, batch_size=200, num_workers=4, drop_last=False, shuffle=False)


# if os.path.exists('submission.csv'):
#     f = open('submission.csv', 'a', newline='')
#     csv_writer = csv.writer(f)
# else:
#     csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
#     csv_writer.writerow(["filename", "label"])


# model.eval()
# with torch.no_grad():
#     for image, filenames in test_dl:
#         image = image_change(image)
#         image = image.to(device)

#         pred = model(image)
# #         pred = torch.argmax(pred, dim=1)
#         label = []
#         for i in range(len(pred)) :
#             character1 = torch.argmax(pred[i][0:36])
#             character2 = torch.argmax(pred[i][36:72])
# #             character3 = torch.argmax(pred[i][72:107])
# #             character4 = torch.argmax(pred[i][107:144])
#             ans = ""
#             ans += alphabet[character1]
#             ans += alphabet[character2]
# #             ans.append(alphabet[character3])
# #             ans.append(alphabet[character4])
#             label.append(ans)
            
#         for i in range(len(filenames)):
#             csv_writer.writerow([filenames[i], label[i]])
#             time.sleep( 0.001 )
            
#     del image, pred

# # for filename, _ in test_data:
# #     if filename.startswith("task1") or filename.startswith("task3"):
# #         csv_writer.writerow([filename, 0])
    
# #     f.close()


In [35]:
torch.cuda.empty_cache()
alphabet = 'abcdefghijklmnopqrstuvwxyz0123456789'
alphabets = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','0','1','2','3','4','5','6','7','8','9']
alpha = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
dic={'alphabet':alphabets}
labelencoder = LabelEncoder()
data_le=pd.DataFrame(dic)
data_le['alphabet'] = labelencoder.fit_transform(data_le['alphabet'])
data_le
onehotencoder = OneHotEncoder()
data_str_ohe=onehotencoder.fit_transform(data_le).toarray()
pd.DataFrame(data_str_ohe)
# data_dum = pd.get_dummies(data_le)
# pd.DataFrame(data_dum)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
class Task3Dataset(Dataset):
    def __init__(self, data, root, transform=None, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task3")]
        print(len(self.data))
#         self.new_data = make_dataset(self.data, alphabet)
#         print(len(self.new_data))
        self.return_filename = return_filename
        self.root = root
        self.transform = transform
#       self.samples = make_dataset(self.data_path,self.ans_path,self.alphabet)
    
    def __getitem__(self, index):
        filename, label = self.data[index]
#         img = cv2.imread(f"{self.root}/{filename}")
        img = io.read_image(f"{self.root}/{filename}")
#         img = cv2.resize(img, (32, 3))
#         img = np.mean(img, axis=2)s
        if self.return_filename:
            return torch.FloatTensor(img / 256), filename
        else:
#             label = torch.Tensor(label)
            t=[]
            for char in str(label):
                v = [0] * 36
                v[alphabet.find(char)] = 1
                t += v
            return torch.FloatTensor(img / 256), torch.Tensor(t)
    def __len__(self):
        return len(self.data)

In [37]:
train_data = []
val_data = []

transfer = transforms.Compose([torch.Tensor()])

with open(f'{TRAIN_PATH}/annotations.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        if random.random() < 0.7:
            train_data.append(row)
        else:
            val_data.append(row)

train_ds = Task3Dataset(train_data, root=TRAIN_PATH, transform=transfer)
train_dl = DataLoader(train_ds, batch_size=32, num_workers=4, drop_last=True, shuffle=True)

val_ds = Task3Dataset(val_data, root=TRAIN_PATH, transform=transfer)
val_dl = DataLoader(val_ds, batch_size=32, num_workers=4, drop_last=False, shuffle=False)

2098
902


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [38]:
class ResNet18_3(nn.Module):
    def __init__(self):
        super(ResNet18_3, self).__init__()

        self.model = models.resnet18(pretrained=True)

        self.model.fc = nn.Linear(512, 144)

    def forward(self, x):
        return self.model(x)

In [39]:
model3 = ResNet18_3().to(device)

torch.cuda.empty_cache()
# print(model)
optimizer = torch.optim.Adam(model3.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
criterion = nn.MultiLabelSoftMarginLoss()

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
image_change = transforms.Compose([transforms.Resize((384, 288)), transforms.Normalize(mean, std)])

for epoch in range(70):
    print(f"Epoch [{epoch}]")
    model3.train()
    for image, label in train_dl:
        image = image_change(image)
        image = image.to(device)
        label = label.to(device)
#         print(label.size())
#         print("Shape of X [N, C, H, W]: ", image.shape)
        pred = model3(image)
#         print(pred.size())
        loss = criterion(pred, label)
#         loss = loss_fn(pred, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del image, label
        
    sample_count = 0
    correct_count = 0
    acc_history = []
    acc_final = 0
    model3.eval()
    with torch.no_grad():
        for image, label in val_dl:
            image = image_change(image)
            image = image.to(device)
            label = label.to(device)

            pred = model3(image)
            loss = criterion(pred, label)
#             loss = loss_fn(pred, label)
            
#             pred = torch.argmax(pred, dim=1)
            
            
#             sample_count += len(image)
#             correct_count += (label == pred).sum()
#             acc = calculat_acc(pred, label)
            pred, label = pred.view(-1, 36), label.view(-1, 36)
            pred = torch.argmax(pred, dim=1)
            label = torch.argmax(label, dim=1)
            pred, label = pred.view(-1, 2), label.view(-1, 2)
            correct = []
            for i, j in zip(pred, label):
                if torch.equal(i, j):
                    correct.append(1)
                else:
                    correct.append(0)
            acc = sum(correct) / len(correct)
            acc_history.append(float(acc))
        del image, pred
    print("accuracy (validation):",  sum(acc_history) / len(acc_history))
#     print("accuracy (validation):", torch.mean(torch.Tensor(acc_history)))
    cur = sum(acc_history) / len(acc_history)
    if cur >= 0.99 :
#         acc_final = cur
#         print(cur)
        torch.save(model3.state_dict(), "/kaggle/working/model_task3")

Epoch [0]
accuracy (validation): 0.0026939655172413795
Epoch [1]
accuracy (validation): 0.010237068965517241
Epoch [2]
accuracy (validation): 0.11099137931034483
Epoch [3]
accuracy (validation): 0.4209770114942529
Epoch [4]
accuracy (validation): 0.6531968390804598
Epoch [5]
accuracy (validation): 0.8230962643678161
Epoch [6]
accuracy (validation): 0.9107399425287356
Epoch [7]
accuracy (validation): 0.9324712643678161
Epoch [8]
accuracy (validation): 0.9572557471264368
Epoch [9]
accuracy (validation): 0.965337643678161
Epoch [10]
accuracy (validation): 0.9642600574712644
Epoch [11]
accuracy (validation): 0.9766522988505748
Epoch [12]
accuracy (validation): 0.9793462643678161
Epoch [13]
accuracy (validation): 0.980962643678161
Epoch [14]
accuracy (validation): 0.977191091954023
Epoch [15]
accuracy (validation): 0.9788074712643678
Epoch [16]
accuracy (validation): 0.9788074712643678
Epoch [17]
accuracy (validation): 0.9847341954022989
Epoch [18]
accuracy (validation): 0.9836566091954023


In [40]:
torch.cuda.empty_cache()
test_data = []
alphabet = 'abcdefghijklmnopqrstuvwxyz0123456789'

with open(f'{TEST_PATH}/../sample_submission.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        test_data.append(row)

test_ds_1 = Task1Dataset(test_data, root=TEST_PATH, return_filename=True)
test_dl_1 = DataLoader(test_ds_1, batch_size=32, num_workers=4, drop_last=False, shuffle=False)
test_ds_2 = Task2Dataset(test_data, root=TEST_PATH, return_filename=True)
test_dl_2 = DataLoader(test_ds_2, batch_size=32, num_workers=4, drop_last=False, shuffle=False)
test_ds_3 = Task3Dataset(test_data, root=TEST_PATH, return_filename=True)
test_dl_3 = DataLoader(test_ds_3, batch_size=32, num_workers=4, drop_last=False, shuffle=False)


if os.path.exists('submission.csv'):
    f = open('submission.csv', 'w', newline='')
    csv_writer = csv.writer(f)
    csv_writer.writerow(["filename", "label"])
else:
    f = open('submission.csv', 'w', newline='')
    csv_writer = csv.writer(f)
    csv_writer.writerow(["filename", "label"])
    
model1 =  ResNet18_1().to(device)
model1.load_state_dict(torch.load("/kaggle/working/model_task1"))
model2 =  ResNet18_2().to(device)
model2.load_state_dict(torch.load("/kaggle/working/model_task2"))
model3 =  ResNet18_3().to(device)
model3.load_state_dict(torch.load("/kaggle/working/model_task3"))

model1.eval()
with torch.no_grad():
    for image, filenames in test_dl_1:
        image = image_change(image)
        image = image.to(device)

        pred = model1(image)
        pred = torch.argmax(pred, dim=1)

        for i in range(len(filenames)):
            csv_writer.writerow([filenames[i], str(pred[i].item())])
            time.sleep( 0.001 )
            
    del image, pred

model2.eval()
with torch.no_grad():
    for image, filenames in test_dl_2:
        image = image_change(image)
        image = image.to(device)

        pred = model2(image)
#         pred = torch.argmax(pred, dim=1)
        label = []
        for i in range(len(pred)) :
            character1 = torch.argmax(pred[i][0:36])
            character2 = torch.argmax(pred[i][36:72])
            ans = ""
            ans += alphabet[character1]
            ans += alphabet[character2]
            label.append(ans)
            
        for i in range(len(filenames)):
            csv_writer.writerow([filenames[i], label[i]])
#             time.sleep( 0.001 )
    del image, pred
    
model3.eval()
with torch.no_grad():
    for image, filenames in test_dl_3:
        image = image_change(image)
        image = image.to(device)

        pred = model3(image)
#         pred = torch.argmax(pred, dim=1)
        label = []
        for i in range(len(pred)) :
            character1 = torch.argmax(pred[i][0:36])
            character2 = torch.argmax(pred[i][36:72])
            character3 = torch.argmax(pred[i][72:108])
            character4 = torch.argmax(pred[i][108:144])
            ans = ""
            ans += alphabet[character1]
            ans += alphabet[character2]
            ans += alphabet[character3]
            ans += alphabet[character4]
            label.append(ans)

        for i in range(len(filenames)):
            csv_writer.writerow([filenames[i], label[i]])
            time.sleep( 0.001 )
            
    del image, pred

# for filename, _ in test_data:
#     if filename.startswith("task1") or filename.startswith("task3"):
#         csv_writer.writerow([filename, 0])
f.close()

2500
1000
